In [ ]:
## leg geometric parameters

from inversegait import JointOffsets,kneelength,hiplength
import numpy as np

LegNames=['Front Right','Front Left','Rear Right','Rear Left']
L1=hiplength # length of hip in mm
L2=kneelength # length of knee in mm

z_rest_foot=-68.92 #resting foot posiition in mm

## Modified Hopf Oscillator CPG-- baswd off Zeng paper 

def hopf_cpg_dot(Q,R,delta,dutycycle,T,b,mu,alpha,gamma,dt):

    # R--> desired gait phase relation between legs
    #b,alpha,gamma,mu= constants
    # T-->period of gait= 1/gaitfrequency
    # delta= transition coefficient, to be integrated for later when the robot will want to shift between gaits
    q_dot=np.zeros(8) #Q=[x1,y1,x2,y2,x3,y3,x4,y4]

    for i in range(4):
        xi=Q[2*i]
        zi=Q[2*i+1]
        q=np.array([[xi],[zi]])
        r2=xi**2+zi**2
        stance_denom=dutycycle*T*(np.exp(-b*zi)+1)
        swing_denom=(1-dutycycle)*T*(np.exp(b*zi)+1)
        omega=np.pi/stance_denom+np.pi/swing_denom
        A=np.array([[alpha*(mu-r2),-omega],[omega,gamma*(mu-r2)]])
        q_dot_first_term=A@q  # covers the first term in equation (9) from Zeng et. al
        q_dot[2*i:2*i+2]=q_dot_first_term.flatten()
    
    # second term
    q_dot += delta * R @ Q  
    Q_new=Q+q_dot*dt
    return Q_new

## the trjaectories are generated for each foot individually
def TrajectoryGeneratorSinewithOffsets(x_hopf,z_hopf,S,H,dutycycle,robotheight,xCOM_shift):
    XX=[]
    ZZ=[]
    
    for i in range(len(x_hopf)):
        phase_rad=np.arctan2(z_hopf[i],x_hopf[i]) #hopf oscillator tells the phase of the leg in radians 
        phase_norm=(phase_rad+np.pi)/(2*np.pi)
        x=S/2*np.cos(2*np.pi*phase_norm)+x_hipoffset+x_COMshift  #S= stride length (mm)

        shifted_phase_norm=(phase_norm+0.5) %1 ## renormalize the phase norm since the original phase normalized has a half a cycle worth of discrepancy
        if shifted_phase_norm < (1-dutycycle):
            z = H * np.sin(2 * np.pi * shifted_phase_norm) #H is swing height (mm), swing phase runs this part
            
        else:
            z = 0  # stance phase
        XX.append(x)
        ZZ.append(z)
        

    return XX,ZZ


# x_abs=S/2*np.cos(2*np.pi*phase_norm)+x_hipoffset+x_COMshift #S= stride length (mm)
#x_COMshift is mainly to indicate the shift seen during gait movement since during analysis it was noticed that the rear hips moved further from their initial position

# z_abs_swing = H * np.sin(2 * np.pi * shifted_phase_norm)+z_robotheightfromground (-ve) +z_rest_footposition (-ve) , H is the clearance of the foot not the swing height (so how much below the neutral position does the foot move)

# z_abs_stance =-H+z_robotheightfromground (-ve) +z_rest_footposition (-ve)